In [1]:
from langchain.chat_models import  ChatOpenAI
from langchain.agents import initialize_agent,AgentType, Tool
from langchain.utilities import SQLDatabase
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List, Optional, Any
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 获取 环境变量
#大模型apikey和url
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

#天气查询的apikey
juheAppKey = os.getenv('juheAppKey')

#数据库信息
mysql_info = os.getenv('mysql_info')

In [3]:
#定义大模型
online_model = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_15916/113483475.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  online_model = ChatOpenAI(


In [4]:
#定义数据库 连接
db = SQLDatabase.from_uri(mysql_info)


In [5]:
#查看数据库中的可用表
db.get_usable_table_names()

['city_stats']

In [6]:
#查看表结构
print(db.get_table_info())


CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/


In [7]:
#查看数据库信息
print(db.get_context())

{'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}


In [8]:


#定义数据库信息查询工具
# class QueryDatabaseInfoInput(BaseModel):
#     query:str = Field(description="查询语句")


class QueryDatabaseInfoTool(BaseTool):
    name:str = Field(default="get_database_info")
    description:str = Field(default="获取数据库信息")
    # args_schema:Type[QueryDatabaseInfoInput] = QueryDatabaseInfoInput

    def _run(self) -> str:
        return db.get_context()
    
    def _arun(self) -> str:
        return "暂时不支持异步查询"
    


#定义数据库表查询工具
class QueryDatabaseTableInput(BaseModel):
    query:str = Field(description="sql查询语句")

class QueryDatabaseTableTool(BaseTool):
    name:str = Field(default="get_database_table")
    description:str = Field(default="根据 sql语句 获取数据库表信息")
    args_schema:Type[QueryDatabaseTableInput] = QueryDatabaseTableInput

    def _run(self, query:str) -> str:
        return db.run(query)

    def _arun(self, query:str) -> str:
        return "暂时不支持异步查询"
    


In [9]:
tools = [QueryDatabaseInfoTool(),QueryDatabaseTableTool()]

In [13]:
agent = initialize_agent(
    tools,
    online_model,
    AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
)

In [14]:
result = agent.run("请帮我查询数据库信息")
result

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_15916/1471654928.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("请帮我查询数据库信息")




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_database_info",
  "action_input": {}
}
```

Observation: {'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}
Thought:I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "数据库信息如下：\n表名：city_stats\n表结构：\ncity_name VARCHAR(255), \npopulation VARCHAR(255), \narea VARCHAR(255)\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821"
}
```

> Finished chain.


'数据库信息如下：\n表名：city_stats\n表结构：\ncity_name VARCHAR(255), \npopulation VARCHAR(255), \narea VARCHAR(255)\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821'

In [17]:
result = agent.run("请帮我查询一下city_stats表中的所有信息")
result



> Entering new AgentExecutor chain...
Thought: 用户希望查询 city_stats 表中的所有信息。我需要使用 get_database_table 工具来执行 SQL 查询语句以获取数据。

Action:
```
{
  "action": "get_database_table",
  "action_input": {
    "query": "SELECT * FROM city_stats"
  }
}
```

Observation: [('北京', '20000000', '400000234'), ('上海', '17689300', '29867200'), ('河北', '19999944', '39999821')]
Thought:我已经获取了 city_stats 表中的所有信息。现在我可以将结果返回给用户。

Action:
```
{
  "action": "Final Answer",
  "action_input": "以下是 city_stats 表中的所有信息：北京，人口20000000，面积400000234；上海，人口17689300，面积29867200；河北，人口19999944，面积39999821。"
}
```

> Finished chain.


'以下是 city_stats 表中的所有信息：北京，人口20000000，面积400000234；上海，人口17689300，面积29867200；河北，人口19999944，面积39999821。'

In [18]:
result = agent.run("请告诉我数据库表city_stats中人口最多的两个城市是哪些，分别是多少人")
result



> Entering new AgentExecutor chain...
Thought: 要回答这个问题，我需要查询数据库表city_stats。我将使用get_database_table工具执行SQL查询来获取人口最多的两个城市及其人口数量。

Action:
```
{
  "action": "get_database_table",
  "action_input": {
    "query": "SELECT city, population FROM city_stats ORDER BY population DESC LIMIT 2;"
  }
}
```


OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'city' in 'field list'")
[SQL: SELECT city, population FROM city_stats ORDER BY population DESC LIMIT 2;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)